# Product Detection

## Import libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2
import itertools
from glob import glob
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

import keras
from keras.models import Model
from keras.layers import Dense
from keras import optimizers
from keras.preprocessing import image
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.applications import InceptionResNetV2

import warnings
warnings.filterwarnings('ignore')

plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = [16, 8]

print('Using Tensorflow version:', tf.__version__)

## Read files

In [ ]:
folder  = '/kaggle/input/shopee-round-2-product-detection-challenge'

train   = pd.read_csv(folder+'/train.csv')
test    = pd.read_csv(folder+'/test.csv')

print("Train size: ", train.shape)
print("Test size: ", test.shape)
print(train.head())

In [ ]:
# path to the train and test folders where all the images are
train_folder='/kaggle/input/shopee-round-2-product-detection-challenge/train/train' 
test_folder ='/kaggle/input/shopee-round-2-product-detection-challenge/test/test'


# combine all train and test images in one path
train_image_path = []

for i in sorted(os.listdir(train_folder)):
    train_image_path.append(glob(os.path.join(train_folder,str(i), "*.jpg")))
    
test_image_path  = glob(os.path.join(test_folder, "*jpg"))

print(len(train_image_path)) # number of categories in train image path
print(len(test_image_path))  # total number of images in test image path

## EDA

In [ ]:
# function to display image by category and by number of images

def display_train_image(path, cat, rangeimage, row, col):
    
    fig, axes = plt.subplots(nrows=row, ncols=col, figsize=(12,10))
    
    for idx in range(rangeimage):
        image      = cv2.imread(path[cat][idx]) 
        image      = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        axes.ravel()[idx].imshow(image)
        #axes.ravel()[idx].axis('off')   
    plt.tight_layout()

def display_test_image(path, rangeimage, row, col):
    
    fig, axes = plt.subplots(nrows=row, ncols=col, figsize=(12,10))
    
    for idx in range(rangeimage):
        image      = cv2.imread(path[idx]) 
        image      = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        axes.ravel()[idx].imshow(image)
        #axes.ravel()[idx].axis('off')   
    plt.tight_layout()


In [ ]:
# product image under first category 

display_train_image(train_image_path, 0, 25, 5, 5) 

In [ ]:
# product image under last category

display_train_image(train_image_path, 41, 25, 5, 5) # last category

In [ ]:
# images under test

display_test_image(test_image_path, 25, 5, 5)

### Separate images to train and validation

In [ ]:
# Configuration
BATCH_SIZE = 128
IMAGE_SIZE = (299,299)

In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255., 
                                   horizontal_flip  = True,
                                   brightness_range = [0.4,0.6],
                                   validation_split = 0.2)

train_generator = train_datagen.flow_from_directory(
    directory   = train_folder,
    target_size = IMAGE_SIZE ,
    color_mode  = "rgb",
    batch_size  = BATCH_SIZE,
    class_mode  = "categorical",
    shuffle     = True,
    seed        = 42,
    subset      = 'training'
)
validation_generator = train_datagen.flow_from_directory(
    directory   = train_folder,
    target_size = IMAGE_SIZE ,
    color_mode  = "rgb",
    batch_size  = BATCH_SIZE,
    class_mode  = "categorical",
    shuffle     = True,
    seed        = 42,
    subset      = 'validation'
)

## Applying Transfer Learning

### InceptionResNetV2

In [ ]:
inceptionresnet = InceptionResNetV2(weights='imagenet',
                                   include_top = True)

In [ ]:
#inceptionresnet.summary()
len(inceptionresnet.layers)

In [ ]:
for layers in (inceptionresnet.layers)[:781]:
    layers.trainable = False

X= inceptionresnet.layers[-2].output
predictions = Dense(42, activation="softmax")(X)
model_final = Model(inputs= inceptionresnet.input, 
                    outputs = predictions)

# model_final.compile(loss = "categorical_crossentropy", 
#                     optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), 
#                     metrics=["accuracy"])


model_final.compile(loss = "categorical_crossentropy", 
                    optimizer = 'adam', metrics=["accuracy"])

In [ ]:
#model_final.load_weights("/kaggle/input/inceptionresnet/inceptionresnet2.h5")

In [ ]:
EPOCHS      = 5

checkpoint1 = ModelCheckpoint('batch1_inceptionresnet1.h5', 
                              monitor='val_loss', 
                              verbose=0, save_best_only=False, 
                              save_weights_only=True, 
                              mode='auto', period=1)




STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size

model_final.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_VALID,
                    callbacks=[checkpoint1],
                    epochs=EPOCHS)

In [ ]:
#model_final.save_weights("inceptionresnet1.h5")

### Predict test category using trained model

In [ ]:
test_datagen = ImageDataGenerator(rescale=1/255.)

test_generator = test_datagen.flow_from_directory(
    directory  = '/kaggle/input/shopee-round-2-product-detection-challenge/test',
    target_size= IMAGE_SIZE,
    color_mode = "rgb",
    batch_size = 1,
    class_mode = None,
    shuffle    = False,
    seed       = 42
)

In [ ]:
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
test_generator.reset()
pred          =model_final.predict_generator(test_generator,
                                             steps=STEP_SIZE_TEST,verbose=1)

predicted_class_indices=np.argmax(pred,axis=1)
labels                 = (train_generator.class_indices)
labels                 = dict((v,k) for k,v in labels.items())
predictions            = [labels[k] for k in predicted_class_indices]
predictions            = list(map(lambda c: str(c).zfill(2), predictions))
filenames              = test_generator.filenames
filenames2             = [file[5:] for file in filenames]

results=pd.DataFrame({"filename":filenames2,
                      "category":predictions})


### Merge predictions with test set

In [ ]:
finalresults = pd.merge(test['filename'], results, how="left", on="filename")

finalresults.to_csv("result_inception2.csv",index=False)

finalresults.head(10)

In [ ]:
finalresults.shape